In [16]:
from minsearch import Index, VectorSearch
from openai import OpenAI, APIConnectionError, RateLimitError, APIStatusError
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
    retry_if_exception_type
)
import pickle
import hashlib
from sentence_transformers import SentenceTransformer

In [3]:
# model for qa purpose in english
embedding_model = SentenceTransformer('multi-qa-distilbert-cos-v1')

In [36]:
openai_client = OpenAI()

@retry(
    wait=wait_random_exponential(multiplier=1, max=60),
    stop=stop_after_attempt(max_attempt_number=5),
    retry=retry_if_exception_type((APIConnectionError, RateLimitError, APIStatusError))
)
def llm_standard(prompt, model='gpt-4o-mini'):
    messages = [
        {"role": "user", "content": prompt}
    ]

    response = openai_client.responses.create(
        model=model,
        input=messages
    )

    return response.output_text

@retry(
    wait=wait_random_exponential(multiplier=1, max=60),
    stop=stop_after_attempt(max_attempt_number=5),
    retry=retry_if_exception_type((APIConnectionError, RateLimitError, APIStatusError))
)
def llm_function_calling(system_prompt: str, query: str, text_search_tool: dict,  model='gpt-4o-mini'):
    chat_messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": query}
    ]

    response = openai_client.responses.create(
        model=model,
        input=chat_messages,
        tools=[text_search_tool]
    )


    return response

# Lets implement hybrid search

In [7]:
FILE_PATH = "vector_search_data.pkl"
loaded_data = None
try:
    with open(FILE_PATH, 'rb') as f:
        loaded_data = pickle.load(f)
        print(f"Data successfully unpickled from {FILE_PATH}")

    # Access the loaded data
    loaded_embeddings = loaded_data['embeddings']
    loaded_docs = loaded_data['documents']
except FileNotFoundError:
    print(f"Error: The file {FILE_PATH} was not found.")
except pickle.UnpicklingError as e:
    print(f"An error occurred during unpickling: {e}")

Data successfully unpickled from vector_search_data.pkl


In [8]:
from minsearch import VectorSearch, Index

In [9]:
index = Index(text_fields=['title', 'description', 'filename', 'section'],
              keyword_fields=[])
index.fit(loaded_docs)

vs = VectorSearch()
vs.fit(loaded_embeddings, loaded_docs)

In [10]:
query = "How can I evaluate classification model results, and ensure numerical data is not drifted?"

In [12]:
text_results = index.search(query=query, num_results=5)
q_v = embedding_model.encode(query)
vector_results = vs.search(query_vector=q_v, num_results=5)

final_results = text_results + vector_results

In [13]:
final_results

[{'title': 'Synthetic data',
  'description': 'Generating test cases and datasets.',
  'filename': 'docs-main/synthetic-data/introduction.mdx',
  'section': '## Use Cases for Synthetic Test Inputs\n\nEvidently Cloud can be utilized for multiple purposes, including:\n\n* **Experiments**: Create test data to see how your LLM application handles various inputs.\n* **Regression Testing**: Validate changes in your AI system before deployment.\n* **Adversarial Testing**: Assess how your system manages tricky or unexpected inputs.\n\nOnce the data is generated, you can evaluate the results using the Evidently Cloud interface or the Evidently Python library.'},
 {'title': 'Synthetic data',
  'description': 'Generating test cases and datasets.',
  'filename': 'docs-main/synthetic-data/introduction.mdx',
  'section': '## Example of Generating Test Inputs\n\nAn illustrative example of how to generate synthetic test inputs can be found in the accompanying GIF, depicting the data generation process

In [39]:
import hashlib

In [25]:
def text_search(query):
    return index.search(query, num_results=5)

def vector_search(query):
    q = embedding_model.encode(query)
    return vs.search(q, num_results=5)

def hybrid_search(query):
    text_results = text_search(query)
    vector_results = vector_search(query)
    
    # Combine and deduplicate results
    seen_ids = set()
    combined_results = []

    for result in text_results + vector_results:
        text_to_hash = result['filename'] + ' ' + result['section'][0:250]
        encoded_string = text_to_hash.encode('utf-8')
        hash_object = hashlib.sha256(encoded_string)
        hex_digest = hash_object.hexdigest()
        if hex_digest not in seen_ids:
            seen_ids.add(result['filename'])
            combined_results.append(result)
    
    return combined_results

In [17]:
hybrid_search(query=query)

[{'title': 'Synthetic data',
  'description': 'Generating test cases and datasets.',
  'filename': 'docs-main/synthetic-data/introduction.mdx',
  'section': '## Use Cases for Synthetic Test Inputs\n\nEvidently Cloud can be utilized for multiple purposes, including:\n\n* **Experiments**: Create test data to see how your LLM application handles various inputs.\n* **Regression Testing**: Validate changes in your AI system before deployment.\n* **Adversarial Testing**: Assess how your system manages tricky or unexpected inputs.\n\nOnce the data is generated, you can evaluate the results using the Evidently Cloud interface or the Evidently Python library.'},
 {'title': 'Synthetic data',
  'description': 'Generating test cases and datasets.',
  'filename': 'docs-main/synthetic-data/introduction.mdx',
  'section': '## Example of Generating Test Inputs\n\nAn illustrative example of how to generate synthetic test inputs can be found in the accompanying GIF, depicting the data generation process

In [46]:
file_path = "vector_search_data.pkl"
try:
    with open(f'{file_path}', 'wb') as f:
        pickle.dump(data_to_save, f)
    print(f"Data successfully pickled and saved to {file_path}")
except pickle.PicklingError as e:
    print(f"An error occurred during pickling: {e}")
except IOError as e:
    print(f"An I/O error occurred: {e}")

Data successfully pickled and saved to vector_search_data.pkl


# Function calling and agentic

In [76]:
text_search_tool = {
    "type": "function",
    "name": "hybrid_search",
    "description": "Search the docs",
    "parameters": {
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "User question to extract answer from the results"
            }
        },
        "required": ["query"],
        "additionalProperties": False
    }
}

In [60]:
system_prompt = """
You are a helpful assistant for finding relevant information from the docs

Use the search tool to find relevant information from the materials before answering questions.

Make multiple searches if needed to provide comprehensive answers.
"""

In [61]:
response = llm_function_calling(system_prompt=system_prompt,
                     query=query,
                     text_search_tool=text_search_tool)

In [62]:
import json

call = response.output[0]

arguments = json.loads(call.arguments)
result = hybrid_search(**arguments)

call_output = {
    "type": "function_call_output",
    "call_id": call.call_id,
    "output": json.dumps(result),
}


In [64]:
chat_messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": query}
]

In [65]:
chat_messages.append(call)
chat_messages.append(call_output)

In [66]:
response = openai_client.responses.create(
    model='gpt-4o-mini',
    input=chat_messages,
    tools=[text_search_tool]
)

print(response.output_text)

In [71]:
from typing import List, Any

In [82]:
def hybrid_search(query: str) -> List[Any]:
    text_results = text_search(query)
    vector_results = vector_search(query)
    
    # Combine and deduplicate results
    seen_ids = set()
    combined_results = []

    for result in text_results + vector_results:
        text_to_hash = result['filename'] + ' ' + result['section'][0:250]
        encoded_string = text_to_hash.encode('utf-8')
        hash_object = hashlib.sha256(encoded_string)
        hex_digest = hash_object.hexdigest()
        if hex_digest not in seen_ids:
            seen_ids.add(result['filename'])
            combined_results.append(result)
    
    return combined_results

In [83]:
text_search_tool = {
    "type": "function",
    "name": "hybrid_search",
    "description": "Search the docs",
    "parameters": {
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "User question to extract answer from the results"
            }
        },
        "required": ["query"],
        "additionalProperties": False
    }
}

In [97]:
from pydantic_ai import Agent, agent

agent = Agent(
    name="faq_agent",
    instructions=system_prompt,
    tools=[hybrid_search],
    model='gpt-4o-mini')

In [98]:
result = await agent.run(user_prompt=query)

In [99]:
print(type(result))

<class 'pydantic_ai.run.AgentRunResult'>


In [100]:
result.new_messages()

[ModelRequest(parts=[UserPromptPart(content='How can I evaluate classification model results, and ensure numerical data is not drifted?', timestamp=datetime.datetime(2025, 12, 27, 3, 52, 35, 295493, tzinfo=datetime.timezone.utc))], timestamp=datetime.datetime(2025, 12, 27, 3, 52, 35, 295762, tzinfo=datetime.timezone.utc), instructions='You are a helpful assistant for finding relevant information from the docs\n\nUse the search tool to find relevant information from the materials before answering questions.\n\nMake multiple searches if needed to provide comprehensive answers.', run_id='e0b84cd1-3bfc-4373-8a59-288ad84997ca'),
 ModelResponse(parts=[ToolCallPart(tool_name='hybrid_search', args='{"query": "evaluate classification model results"}', tool_call_id='call_TOcbGjl3LuwyFqBFLIgZNUpI'), ToolCallPart(tool_name='hybrid_search', args='{"query": "numerical data drift detection"}', tool_call_id='call_3b0pLvq4yoNVD02GMbjf8WHj'), ToolCallPart(tool_name='hybrid_search', args='{"query": "moni